# Аутентификация

## Подготовка

### Создание топика

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-listeners-demo-topic \
    --create \
    --partitions 1 \
    --replication-factor 1

### Получение списка топиков

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list

### Записать данные в топик

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-listeners-demo-topic \
    <<<$(yes 'Hello, World!' 2>/dev/null | head -n 10)

## Чтение топика с аутентификацией и без

Каждый брокер в текущем кластере открывает 4 порта:
- 9092 - для клиентских запросов без аутентификации
- 19092 - для клиентских запросов с [SASL](https://ru.wikipedia.org/wiki/Simple_Authentication_and_Security_Layer) аутентифкацией
- 29092 - для общения брокеров друг с другом (репликация партиций топиков)
- 39092 - для общения брокеров с `zookeeper`

### Чтение из топика без аутентификации

По порту `9092` чтение данных доступно без ограничений:

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":9092 \
    --topic my-listeners-demo-topic \
    --from-beginning \
    --timeout-ms 10000

### Чтение из топика c SASL аутентификацией

Чтение данных по порту `19092` требует аутентификации:

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":19092 \
    --topic my-listeners-demo-topic \
    --from-beginning \
    --timeout-ms 5000

Можно заметить:

- на печать несколько раз выводилось сообщение `disconnected`. Брокер `kafka1` не пропускал запросы к порту `19092`, т.к. клиент не прошел аутентификацию;
- ни одного сообщения из топика не было прочитано: `Processed a total of 0 messages`.

Чтобы выполнять запросы на `kafka1:19092`, необходимо выполнить аутентификацию. В данном случае настроена [`JAAS`](https://docs.oracle.com/javase/8/docs/technotes/guides/security/jaas/JAASRefGuide.html) аутентификация `SASL_PLAINTEXT`, что означает:

- `SASL` - клиент должен предоставить `username` и `password`;
- `PLAINTEXT` - все сообщения будут передаваться и храниться в открытом виде (без шифрования).

Для аутентификации необходимо:

1. создать файл `/tmp/auth.properties` с конфигурацией консьюмера:
    - указать какой протокол аутентификации установлен на брокере (`SASL_PLAINTEXT`);
    - указать какой механизм `SASL` установлен для клиентских подключений;
    - указать `jaas` конфигурацию с `username` и `password`.
2. указать файл `/tmp/auth.properties` в ключе `--consumer.config`.

In [ ]:
HOST=${KAFKA_HOST} \
new_file /tmp/auth.properties <<EOF
security.protocol=SASL_PLAINTEXT
sasl.mechanism=PLAIN
sasl.jaas.config=org.apache.kafka.common.security.plain.PlainLoginModule required username="alice" password="alice";
EOF

HOST=${KAFKA_HOST} execute \
cat /tmp/auth.properties

После аутентификации чтение сообщений становится доступным:

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":19092 \
    --topic my-listeners-demo-topic \
    --from-beginning \
    --timeout-ms 10000 \
    --consumer.config /tmp/auth.properties

### Запись в топик с аутентификацией

Для записи в топик необходимо указать файл с аутентификацией через `--producer.config` ключ:

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":19092 \
    --producer.config /tmp/auth.properties \
    --topic my-listeners-demo-topic \
    <<<$(yes 'Привет, мир!' 2>/dev/null | head -n 5)

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":19092 \
    --topic my-listeners-demo-topic \
    --from-beginning \
    --timeout-ms 10000 \
    --consumer.config /tmp/auth.properties

При проектировании кластера необходимо в самом начале подумать об безопасности кластера. Apache Kafka [позволяет](https://kafka.apache.org/documentation/#security) установить более строгие требования к аутентификации, авторизации и шифрованию. Apache Kafka позволяет сменить схему аутентификации и шифрования в будущем, но это будет сопряжено с трудностями бизнес характера.

### Выводы

1. Настройка безопасности в Apache Kafka является опциональной;
1. Apache Kafka позволяет очень точно настраивать обработку траффика в кластере;
1. Каждый тип трафика направляется на свой собственный порт;
1. Каждый порт может выставлять требования по аутентификации, авторизации и шифрованию.

### Задания

1. Для чего нужно выделять два и более порта для чтения и записи данных в Apache Kafka?
1. Внести изменения в конфигурацию кластера, чтобы можно было выполнять логин со своим пользователем, `username` которого является вашим именем, например, `nikita`;
1. Запустить консольный консьюмер, указав своего пользователя и его пароль в параметрах аутентификации;
1. Запустить консольный продюсер, указав своего пользователя и его пароль в параметрах аутентификации;
1. Настроить `SASL_PLAINTEXT` на порту `29092`.

## Авторизация ACL

_Авторизация_ - процесс проверки (подтверждения) прав при попытке выполнения определенных действий

Авторизация в Apache Kafka выполняется на базе ACL - Access Ccontrol List. Администратор должен явно написать какой пользователь какие имеет права при работе с конкретным объектом. Управлять списком прав доступа можно при помощи [`kafka-acl`](https://docs.confluent.io/kafka/operations-tools/kafka-tools.html#kafka-acls).

### Настройка доступа к топику

Если к топику применяется авторизация на базе ACL, то топик становится недоступным для всех, кто находится вне ACL топика:

#### Создать топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-acl-demo-topic \
    --create \
    --partitions 1 \
    --replication-factor 1

#### Запись данных в топик

На текущий момент топик никак не защищен, а поэтому в него можно писать данные:

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-acl-demo-topic \
<<EOF
Отправка пред настройкой ACL
EOF

#### Чтение данных

На текущий момент топик никак не защищен, а поэтому из него можно читать данные:

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-acl-demo-topic \
    --from-beginning \
    --timeout-ms 10000

#### Настройка ACL

После настройки ACL возможность писать и читать данные из топика становится недоступной:

In [ ]:
kafka-acls --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
  --add \
  --allow-principal User:alice \
  --operation read \
  --operation write \
  --operation alter \
  --operation delete \
  --topic my-acl-demo-topic

Получение ACL списка для топика:

In [ ]:
kafka-acls --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list \
    --topic my-acl-demo-topic

Тут можно отметить:

1. имя пользователя (`User:alice`) должно быть известно изначально (файл `/etc/kafka/kafka_jaas.conf`);
1. параметр `--operation` можно повторять столько раз, сколько прав нужно выдать.

Текущая конфигурация аутентификации:

In [ ]:
HOST=$KAFKA_HOST execute \
cat /etc/kafka/kafka_jaas.conf

#### Запись в топик c ACL

После настройки ACL на топике, из топика невозможно ни читать, ни писать в него, если не выполнить авторизацию, т.е. указать имя пользователя из ACL:

1. Нельзя записать даже на незащищенном порту (9092)

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":9092 \
    --topic my-acl-demo-topic \
<<EOF
Невозможно записать по открытому порту
EOF

2. Нельзя записать, если не указан пользователь из ACL:

In [ ]:
HOST=$KAFKA_HOST \
new_file /tmp/admin.properties <<EOF
security.protocol=SASL_PLAINTEXT
sasl.mechanism=PLAIN
sasl.jaas.config=org.apache.kafka.common.security.plain.PlainLoginModule required username="admin" password="admin";
EOF

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":19092 \
    --producer.config /tmp/admin.properties \
    --topic my-acl-demo-topic \
<<EOF
У admin так же нет прав на запись
EOF

3. Успешная запись под пользователем `alice`, который находится в ACL списке

In [ ]:
HOST=$KAFKA_HOST \
new_file /tmp/alice.properties <<EOF
security.protocol=SASL_PLAINTEXT
sasl.mechanism=PLAIN
sasl.jaas.config=org.apache.kafka.common.security.plain.PlainLoginModule required username="alice" password="alice";
EOF

In [ ]:
kafka-console-producer --bootstrap-server "$KAFKA_HOST":19092 \
    --producer.config /tmp/alice.properties \
    --topic my-acl-demo-topic  \
<<EOF
alice находится в ACL
alice может писать в топик my-acl-demo-topic
EOF

#### Чтение из топика c ACL

1. Ошибка авторизации при чтении данных по незащищенному порту (9092)

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":9092 \
    --topic my-acl-demo-topic \
    --from-beginning \
    --timeout-ms 10000

2. Пользователи не из ACL спсика не могут читать сообщения из защищенного топика:

In [ ]:
HOST=$KAFKA_HOST execute \
cat /tmp/admin.properties

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":19092 \
    --topic my-acl-demo-topic \
    --from-beginning \
    --timeout-ms 10000 \
    --consumer.config /tmp/admin.properties

3. Успешное чтение для пользователя из ACL списка:

In [ ]:
HOST=$KAFKA_HOST execute \
cat /tmp/alice.properties

In [ ]:
kafka-console-consumer --bootstrap-server "$KAFKA_HOST":19092 \
    --topic my-acl-demo-topic \
    --from-beginning \
    --timeout-ms 10000 \
    --consumer.config /tmp/alice.properties

### Выводы

1. Авторизация настраивается при помощи ACL - Access Control Lists;
1. Авторизация настраивается для конкретного пользователя на конкретный топик для выполнения конкретных действий;
1. Если к топику применены настройки авторизации, то только пользователи из ACL списка могут получать доступ к топику;
1. Перед авторизацией обязательно необходимо выполнить аутентификацию, чтобы проверить права пользователя.

### Задание

1. Назначить права доступа на чтение топика `my-acl-demo-topic` своему пользователю;
1. Убедиться, что читать из топика можно, а записывать нельзя;
1. Выдать права на запись в топик `my-acl-demo-topic`;
1. Убедиться, что есть доступ на запись данных:
    - найти своего пользователя в ACL при помощи `kafka-acl`;
    - прочитать данные из топика `my-acl-demo-topic` при помощи `kafka-console-consumer`.

## Удалить топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":19092 \
    --topic my-acl-demo-topic \
    --delete \
    --command-config /tmp/alice.properties

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":19092 \
    --list \
    --command-config /tmp/alice.properties